In [18]:
from astropy.io import fits
from astropy.table import Table
from astropy.cosmology import Planck13
from astropy import units as u
from astropy.coordinates import SkyCoord
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from tqdm import tqdm

In [19]:
!ls

 2D_smooth_01_s_03.png
'coma_MGS_RaDecZ_XYZ:0.00|0.06.csv'
'coma_MGS_RaDecZ_XYZ:0.01|0.04.csv'
 _disperse
 _disperse_02
 DisPerSe.ipynb
 disperse_MGS_coma_2D.ipynb
 disperse_MGS_coma_cart.ipynb
 disperse_MGS_coma.ipynb
 disperse_sdss7_coma.ipynb
 DR5_cluster-catalog_v1.1.fits
 MGS
'MGS_2D_smooth:1_s:3_board:smooth_comaZ:0.006|0.04'
 MGS_coma_2D_ascii.txt
 README.md
 SDSS
 visual.ipynb


In [20]:
!ls MGS

main_gals-DR123a.fit  main_gals-DR5.fit  main_gals-EXTRA.fit
main_gals-DR123b.fit  main_gals-DR6.fit  main_gals-SPECIALa.fit
main_gals-DR4.fit     main_gals-DR7.fit  main_gals-SPECIALb.fit


In [21]:
folder = 'MGS'
files = [
    'main_gals-DR123a.fit', 
    'main_gals-DR123b.fit',
    'main_gals-DR4.fit',
    'main_gals-DR5.fit',
    'main_gals-DR6.fit',
    'main_gals-DR7.fit'
]

In [22]:
t = Table.read(folder+'/'+files[0], format='fits')
sorted(list(t.columns))

['ab_dev',
 'ab_deverr',
 'ab_exp',
 'ab_experr',
 'b',
 'blueslope',
 'camcol',
 'catid',
 'class',
 'colc',
 'colcerr',
 'colv',
 'colverr',
 'contchi2',
 'counts_dev',
 'counts_deverr',
 'counts_exp',
 'counts_experr',
 'counts_model',
 'counts_modelerr',
 'dec',
 'deriv2',
 'dev_l',
 'dev_lnl',
 'devaucl',
 'diag_dec',
 'diag_primtarget',
 'diag_ra',
 'diag_sectarget',
 'eclass',
 'ecoeff1',
 'ecoeff2',
 'ecoeff3',
 'ecoeff4',
 'ecoeff5',
 'eta',
 'exp_l',
 'exp_lnl',
 'expl',
 'fibercounts',
 'fibercountserr',
 'fiberid',
 'field',
 'firstdelta',
 'firsteta',
 'firstid',
 'firstint',
 'firstlambda',
 'firstmajor',
 'firstmatch',
 'firstminor',
 'firstpa',
 'firstpeak',
 'firstrms',
 'flags',
 'flags2',
 'fracpsf',
 'galclass',
 'griflux',
 'grisn',
 'holetype',
 'id',
 'iso_a',
 'iso_aerr',
 'iso_agrad',
 'iso_b',
 'iso_berr',
 'iso_bgrad',
 'iso_colc',
 'iso_colcerr',
 'iso_colcgrad',
 'iso_phi',
 'iso_phierr',
 'iso_phigrad',
 'iso_rowc',
 'iso_rowcerr',
 'iso_rowcgrad',
 'l',
 

In [23]:
dfs = []
for file in files:
    data = Table.read(folder+'/'+file, format='fits')
    df = data[['ra', 'dec', 'zfinal', 'zconffinal', 'zwarning']].to_pandas()
    dfs.append(df)

In [24]:
full = pd.concat(dfs)

In [25]:
full

,ra,dec,zfinal,zconffinal,zwarning
0,146.714217,-1.041278,0.021265,0.998093,0
1,146.744138,-0.652219,0.203789,0.999300,0
2,146.628573,-0.765148,0.064672,0.998981,0
3,146.631654,-0.988261,0.052667,0.978025,0
4,146.919453,-0.990526,0.213949,0.984952,0
...,...,...,...,...,...
112196,260.743174,31.944035,0.033893,0.999611,0
112197,260.800576,31.898992,0.159699,0.999659,0
112198,260.716009,32.024276,0.167412,0.999942,0
112199,260.697236,32.285628,0.111586,0.999824,0


In [26]:
filtered = full.loc[full['zwarning'] == 0]
filtered = filtered.loc[filtered['zconffinal'] > 0.35]
filtered = filtered.loc[filtered['zfinal'] > 0.0]
filtered = filtered.loc[filtered['ra'] > 90]
filtered = filtered.loc[filtered['ra'] < 300]
# filtered = filtered.loc[filtered['dec'] > 0]
filtered = filtered.loc[(filtered['dec'] + 1.35*filtered['ra']-400) < 0]
filtered

,ra,dec,zfinal,zconffinal,zwarning
0,146.714217,-1.041278,0.021265,0.998093,0
1,146.744138,-0.652219,0.203789,0.999300,0
2,146.628573,-0.765148,0.064672,0.998981,0
3,146.631654,-0.988261,0.052667,0.978025,0
4,146.919453,-0.990526,0.213949,0.984952,0
...,...,...,...,...,...
112196,260.743174,31.944035,0.033893,0.999611,0
112197,260.800576,31.898992,0.159699,0.999659,0
112198,260.716009,32.024276,0.167412,0.999942,0
112199,260.697236,32.285628,0.111586,0.999824,0


In [27]:
# fig = plt.figure(figsize=(18, 12))
# plt.scatter(filtered[['ra']], filtered[['dec']], s=2)
# x = np.linspace(100.,300.)
# k, b = -1.35, 400
# plt.plot(x, k*x+b, color='r')
# plt.xlim(100, 300)
# plt.ylim(-10, 80)

In [28]:
filtered_rdz = filtered[['ra', 'dec', 'zfinal']]
filtered_rdz.columns = ['RA', 'DEC', 'Z']
filtered_rdz

,RA,DEC,Z
0,146.714217,-1.041278,0.021265
1,146.744138,-0.652219,0.203789
2,146.628573,-0.765148,0.064672
3,146.631654,-0.988261,0.052667
4,146.919453,-0.990526,0.213949
...,...,...,...
112196,260.743174,31.944035,0.033893
112197,260.800576,31.898992,0.159699
112198,260.716009,32.024276,0.167412
112199,260.697236,32.285628,0.111586


In [29]:
coma_ra_int = (120, 280)
coma_dec_int = (-20, 80)
coma_z_int = (0.000, 0.060)
# coma_z_int = (0.006, 0.040)

coma = filtered_rdz[
        (coma_ra_int[0] < filtered_rdz['RA']) & (filtered_rdz['RA'] < coma_ra_int[1]) & \
        (coma_dec_int[0] < filtered_rdz['DEC']) & (filtered_rdz['DEC'] < coma_dec_int[1]) & \
        (coma_z_int[0] < filtered_rdz['Z']) & (filtered_rdz['Z'] < coma_z_int[1])
    ]
coma.reset_index(drop=True, inplace=True)

In [30]:
coma

,RA,DEC,Z
0,146.714217,-1.041278,0.021265
1,146.631654,-0.988261,0.052667
2,146.963904,-0.545017,0.056071
3,147.176391,-0.354031,0.006325
4,147.329503,0.028901,0.048089
...,...,...,...
108526,261.103501,31.590881,0.025233
108527,261.147576,31.163356,0.047247
108528,261.246269,31.260667,0.046046
108529,260.714751,32.361811,0.054565


In [31]:
CX = []
CY = []
CZ = []
cosmo = Planck13
for i in tqdm(range(coma.shape[0])):
    c = SkyCoord(
        ra=coma.iloc[i]['RA']*u.degree, 
        dec=coma.iloc[i]['DEC']*u.degree,
        distance=cosmo.comoving_distance(coma.iloc[i]['Z']),
        frame='fk5'
    )
    c.representation_type = 'cartesian'
    CX.append(c.x.value)
    CY.append(c.y.value)
    CZ.append(c.z.value)

100%|██████████| 108531/108531 [01:51<00:00, 973.96it/s]


In [32]:
coma = coma.assign(CX=CX)
coma = coma.assign(CY=CY)
coma = coma.assign(CZ=CZ)
coma

,RA,DEC,Z,CX,CY,CZ
0,146.714217,-1.041278,0.021265,-78.234023,51.362363,-1.701025
1,146.631654,-0.988261,0.052667,-192.149214,126.546807,-3.968854
2,146.963904,-0.545017,0.056071,-205.200731,133.442783,-2.328442
3,147.176391,-0.354031,0.006325,-23.476451,15.143235,-0.172623
4,147.329503,0.028901,0.048089,-177.062120,113.543151,0.106100
...,...,...,...,...,...,...
108526,261.103501,31.590881,0.025233,-14.617892,-93.385240,58.129839
108527,261.147576,31.163356,0.047247,-27.218890,-174.765495,106.963183
108528,261.246269,31.260667,0.046046,-26.213694,-170.239373,104.565174
108529,260.714751,32.361811,0.054565,-32.477262,-198.647500,127.551154


In [33]:
coma.to_csv('coma_MGS_RaDecZ_XYZ:0.00|0.06.csv', index=False)

In [ ]:
coma = pd.read_csv('coma_MGS_RaDecZ_XYZ:0.00|0.06.csv')
coma_ra_int = (130, 260)
coma_dec_int = (-10, 70)
coma_z_int = (0.010, 0.040)
# coma_z_int = (0.006, 0.040)

coma = filtered_rdz[
        (coma_ra_int[0] < filtered_rdz['RA']) & (filtered_rdz['RA'] < coma_ra_int[1]) & \
        (coma_dec_int[0] < filtered_rdz['DEC']) & (filtered_rdz['DEC'] < coma_dec_int[1]) & \
        (coma_z_int[0] < filtered_rdz['Z']) & (filtered_rdz['Z'] < coma_z_int[1])
    ]

In [ ]:
with open('MGS_coma_ascii.txt', 'w') as coma_f:
    coma_f.write('# ra dec z\n')
    for i in range(coma.shape[0]):
        t = coma.iloc[i]
        coma_f.write(f'{t.RA}\t{t.DEC}\t{t.Z}\n')

In [ ]:
SIGMA = 3
SMOOTH = 1
BOARD = 'smooth'

In [ ]:
!_disperse_02/bin/delaunay_3D MGS_coma_ascii.txt -btype {BOARD} -smooth {SMOOTH}

In [ ]:
!_disperse_02/bin/mse MGS_coma_ascii.txt.NDnet -upSkl -nsig {SIGMA}

In [ ]:
!_disperse_02/bin/skelconv MGS_coma_ascii.txt.NDnet_s{SIGMA}.up.NDskl -toRaDecZ -to NDskl_ascii

In [ ]:
def read_skl_ascii_RaDecZ(file_name):
    cps = []
    fils = []
    with open(file_name) as f:
        s = ''
        while s != '[CRITICAL POINTS]':
            s = f.readline().strip()
        cp_num = int(f.readline().strip())
        for i in range(cp_num):
            cp = {}
            type_, ra, dec, z, _, _, _ = tuple(map(float, f.readline().split()))
            cp['RA'] = ra
            cp['DEC'] = dec
            cp['Z'] = z
            cp['type'] = int(type_)
            cps.append(cp)
            for i in range(int(f.readline())):
                f.readline()
        
        while s != '[FILAMENTS]':
            s = f.readline().strip()
        fil_num = int(f.readline())
        for i in range(fil_num):
            fil = {}
            cp1, cp2, sp_num = tuple(map(int, f.readline().split()))
            fil['CP1_id'] = cp1
            fil['CP2_id'] = cp2
            fil['sample_points'] = []
            for j in range(sp_num):
                fil['sample_points'].append(tuple(map(float, f.readline().split())))
            fils.append(fil)
            
    return cps, fils

In [ ]:
cps, fils = read_skl_ascii_RaDecZ(f'MGS_coma_ascii.txt.NDnet_s{SIGMA}.up.NDskl.RaDecZ.a.NDskl')

In [ ]:
!rm MGS_coma_ascii.txt.* test_smooth.dat

In [ ]:
cps[0]

In [ ]:
fils[0]

In [ ]:
font = {'size': 16}
plt.rc('font', **font)
fig = plt.figure(figsize=(18, 12))

plt.scatter(coma['RA'], coma['DEC'], c='grey', s=8)

d = {4: 'xkcd:brown', 3: 'red', 2: 'green', 1: 'orange', 0: 'blue'}
x = []
y = []
c = []
for cp in cps:
    x.append(cp['RA'])
    y.append(cp['DEC'])
    c.append(d[cp['type']])
plt.scatter(x, y, c=c, s=100)
    

# for fil in tqdm(fils):
#     points = fil['sample_points']
#     x = []
#     y = []
#     for i in range(len(points)):
#         x.append(points[i][0])
#         y.append(points[i][1])
#     plt.plot(x, y, 'b', linewidth=1)
ax = fig.get_axes()
ax[0].invert_xaxis()
plt.title(f'MGS_smooth:{SMOOTH}_s:{SIGMA}_board:{BOARD}_comaZ:{coma_z_int[0]}|{coma_z_int[1]}')
plt.savefig(f'MGS_smooth:{SMOOTH}_s:{SIGMA}_board:{BOARD}_comaZ:{coma_z_int[0]}|{coma_z_int[1]}', format='jpg')

In [ ]:
coma

In [ ]:
coma_center_ra = 195
coma_center_dec = 28
coma_center_z = 0.023

In [ ]:
from astropy.cosmology import Planck13, z_at_value
cosmo = Planck13

In [ ]:
cosmo.H0

In [ ]:
cosmo.comoving_distance(coma_center_z)

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord

In [ ]:
c = SkyCoord(
    ra=coma_center_ra*u.degree, 
    dec=coma_center_dec*u.degree,
    distance=cosmo.comoving_distance(coma_center_z),
    frame='fk5'
#     z=coma_center_z
)
c

In [ ]:
c.representation_type

In [ ]:
c.representation_type = 'cartesian'
c

In [ ]:
c.representation_type = 'spherical'
c

In [ ]:
c = SkyCoord(
    x=-86.30994466,
    y=-23.12667997,
    z=47.51069945,
    frame='fk5',
    unit='Mpc',
    representation_type='cartesian'
)
c

In [ ]:
c.representation_type = 'spherical'
c

In [ ]:
c.ra.value

In [ ]:
c.distance

In [ ]:
z_at_value(cosmo.comoving_distance, c.distance)